# This project explores the construction of 2-out-of-3-OT from 1-of-2-OT

Here we make $T$ parallel calls to the OT oracles.

We start with $T = 3$

## Problem representation:

$C_1,C_2,C_3$ are the client's valid inputs.

From client's correctness, each 6-tuple of bits is assigned to some $(x1,x2,x3)$ triple.

The assignments should create an independent set in a graph with $2^{2T+3}$ veritces.

There is an edge between $v_g$ and ${v'}_{g'}$ iff. $g$ and $g'$ differ of a pair $(i,j)=[3]\setminus \{k\}$ of bits, but $v[c_k] \ne v'[c_k]$.

In [6]:
import itertools
import pprint
import random

# TODO: change hardcoded 3 into T everywhere in the code

def tup_2_num(g):
    (ind, v_arr) = g
    return 256*ind[0] + 128*ind[1] + 64*ind[2] + v_arr[2][1] + 2*v_arr[2][0] + 4*v_arr[1][1] + 8*v_arr[1][0] + 16*v_arr[0][1] + 32*v_arr[0][0]

def change_one(pair, index, new_value):
    """Return new tuple with one element modified."""
    new_pair = list(pair)
    new_pair[index] = new_value
    return tuple(new_pair)

def product(*args, **kwargs):
    return list(itertools.product(*args, **kwargs))

# just check the dimension of the distribution where (0,x,y) - (1,x,y) mathch completely, and 
# the other two dimensions match by (1,1,0) and (1,1,1) respectively
def calc_dimension():
    # we in fact need only the (0,*,*) dimensions. The rest are identical and do no change anything
    L = []
    t = 0
    for (a1,a2,a3,v1,v2,v3,b1,b2,v3p,a1p,a2p,a3p,b1p,b2p) in itertools.product(range(2), repeat = 14):
        v = random.randint(0,25)
        if (v == 0):
            print 'step ',t
            cur = [0 for i in range(512)]
            cur[ tup_2_num(((0,0,0),((a1,v1),(a2,v2),(v3,a3))))] = 1
            cur[tup_2_num(((0,0,1),((b1,v1),(b2,v2),(v3p,a3))))] = 1
            cur[tup_2_num(((0,1,0),((a1p,v1),(a2p,v2),(v3,a3p))))] = 1
            cur[tup_2_num(((0,1,1),((b1p,v1),(b2p,v2),(v3p,a3p))))] = 1
            L.append(cur)
        t = t+1
    m = matrix(QQ,L)
    print 'rank(m) = ',m.rank()                  
                      
                      
# v_arr has the right value at c projection fixed
def add_constraints1(constraint_list, client_index, v_ind1, v_ind2, v_arr):
     weights = {}
     v_ind_list = [v_ind1, v_ind2]
     w = [1, -1] 
     L = [0 for i in range(512)]   
     for i in range(2):
         v_ind = v_ind_list[i]
         weight = w[i]   
         for rest in itertools.product(range(2), repeat = 3):
             v_arr_new = tuple([change_one(pair, c^1, r)
                                        for (pair, c, r) in zip(v_arr, client_index, rest)])
             L[tup_2_num((v_ind, v_arr_new))] = weight
     constraint_list.append(tuple(L + [0]))
     constraint_list.append(tuple([-x for x in L] + [0]))   
        
def add_constraints2(constraint_list, v_ind):
    L = [0 for i in range(512)]
    for v_arr in product(list(product(range(2), repeat = 2)), repeat = 3):
        L[tup_2_num((v_ind, v_arr))] = -1
    constraint_list.append(tuple(L + [1]))
    constraint_list.append(tuple([-x for x in L] + [-1]))
    # print 'added according to L = ',L    
    # p.add_constraint(sum(lv[o] for o in L) <= 1)
    # p.add_constraint(sum(-lv[o] for o in L) <= -1)

def add_var_constraints(constraint_list):
    # add a 0 at the end
    L = [0 for i in range(513)]
    for i in range(512):
        L[i] = 1
        constraint_list.append(tuple(L))
        L[i] = 0
    
# each of the list elements is a non-empty list of triples
def generate_LP(client_index_list):
    print 'generating LP..'
    constraint_list = []
    
    # type 1 constraints
    for k in range(3):
        for (x,y) in itertools.product(range(2), repeat = 2):
            for client_index in c_lists[k]:
                for v_arr_c in itertools.product(range(2), repeat = 3):
                    v_arr = [change_one((0, 0), c, r) for (c, r) in zip(client_index, v_arr_c)]
                    v1_ind = [0,0,0]
                    v2_ind = [0,0,0]
                    v1_ind[k] = 0
                    v2_ind[k] = 1
                    v1_ind[(k+1) % 3] = v2_ind[(k+1) % 3] = x
                    v1_ind[(k+2) % 3] = v2_ind[(k+2) % 3] = x  
                    add_constraints1(constraint_list, client_index, tuple(v1_ind), tuple(v2_ind), v_arr)

    # type 2 constraints
        
    for v_ind in itertools.product(range(2), repeat = 3):
        add_constraints2(constraint_list, v_ind)
        
    add_var_constraints(constraint_list)
    p = Polyhedron(constraint_list)
    for q in p.Vrepresentation():
        print(q)
   
def main1():
    print 'starting up'
    vectors = list(itertools.product(range(2), repeat = 3))
    print 'vectors = ', vectors
    spread_indices = list(itertools.product(range(3), repeat = 5))
    # print 'spread_indices = ', spread_indices
    nsteps = 0
    for i in spread_indices:
        print 'started with i = ',i
        flag1 = (1 in i)
        flag2 = (1 in i)
        endings = []
        print 'flags =',
        if flag1 and flag2:
            endings = list(itertools.product(range(3), repeat = 2))
        elif flag1 and not flag2:
            endings = [(2,0),(2,1),(2,2),(0,2),(1,2)]
        elif flag2 and not flag1:
            endings = [(1,0),(1,1),(1,2),(0,1),(2,1)]
        else:
            endings = [(1,2),(2,1)]
        print 'went over endings in ',i,' = ',endings    
        for j in endings:
            c_distr = i + j
            c_lists = [[(0,0,0)],[],[]]
            for (t,d) in enumerate(c_distr):
                [d].append(vectors[t+1])
            print 'c distribution = ',c_distr    
            generate_LP(c_lists)
            nsteps = nsteps + 1
        if nsteps > 10:
            break
                      
calc_dimension()                      

step  45
step  100
step  101
step  107
step  115
step  118
step  130
step  142
step  225
step  243
step  258
step  273
step  315
step  320
step  361
step  389
step  467
step  501
step  548
step  565
step  613
step  616
step  642
step  647
step  650
step  661
step  728
step  748
step  787
step  814
step  833
step  866
step  881
step  955
step  966
step  985
step  1005
step  1006
step  1007
step  1014
step  1035
step  1047
step  1073
step  1090
step  1098
step  1151
step  1174
step  1179
step  1183
step  1242
step  1246
step  1249
step  1280
step  1308
step  1322
step  1366
step  1451
step  1459
step  1463
step  1474
step  1509
step  1534
step  1549
step  1613
step  1723
step  1755
step  1816
step  1835
step  1837
step  1859
step  1910
step  1932
step  1957
step  1962
step  1971
step  2072
step  2109
step  2115
step  2117
step  2151
step  2155
step  2162
step  2213
step  2272
step  2274
step  2296
step  2308
step  2314
step  2318
step  2324
step  2349
step  2353
step  2371
step  2390
ste

step  5872
step  5919
step  5953
step  5996
step  5998
step  6006
step  6030
step  6035
step  6050
step  6068
step  6075
step  6076
step  6080
step  6082
step  6108
step  6161
step  6171
step  6218
step  6221
step  6229
step  6271
step  6296
step  6336
step  6346
step  6388
step  6390
step  6445
step  6496
step  6508
step  6522
step  6595
step  6609
step  6632
step  6652
step  6670
step  6673
step  6691
step  6727
step  6729
step  6740
step  6747
step  6782
step  6874
step  6938
step  6962
step  6981
step  7004
step  7042
step  7071
step  7108
step  7111
step  7119
step  7136
step  7151
step  7156
step  7163
step  7181
step  7192
step  7199
step  7238
step  7241
step  7261
step  7273
step  7296
step  7378
step  7408
step  7415
step  7420
step  7484
step  7539
step  7554
step  7592
step  7623
step  7636
step  7655
step  7678
step  7720
step  7731
step  7735
step  7789
step  7897
step  7925
step  7949
step  7968
step  8016
step  8018
step  8026
step  8055
step  8071
step  8122
step  8144


step  11413
step  11428
step  11432
step  11435
step  11443
step  11480
step  11483
step  11490
step  11520
step  11525
step  11533
step  11580
step  11595
step  11608
step  11635
step  11650
step  11658
step  11710
step  11712
step  11740
step  11775
step  11816
step  11839
step  11842
step  11847
step  11884
step  11892
step  11900
step  11933
step  11935
step  11940
step  11972
step  11974
step  11975
step  12023
step  12028
step  12059
step  12102
step  12106
step  12130
step  12137
step  12155
step  12168
step  12190
step  12224
step  12239
step  12340
step  12344
step  12376
step  12377
step  12410
step  12446
step  12452
step  12480
step  12482
step  12572
step  12602
step  12607
step  12615
step  12620
step  12643
step  12644
step  12681
step  12714
step  12750
step  12774
step  12780
step  12787
step  12802
step  12864
step  12880
step  12895
step  12899
step  12932
step  12944
step  12951
step  12964
step  13012
step  13062
step  13104
step  13132
step  13183
step  13251
ste

16318
step  16372
rank(m) =  228


In [1]:
p = MixedIntegerLinearProgram()
v = p.new_variable(real=True, nonnegative=True)
p.solve()
x, y, z = v['x'], v['y'], v['z']
type(2*x <= 0)
v = p.get_values(v)
print v.values()
print v.keys()
p = Polyhedron(ieqs = [(1,2,4), (3,5,7), (4,20,9)])
for q in p.Vrepresentation():
    print(q)
print p

[0.0, 0.0, 0.0]
['y', 'x', 'z']
A ray in the direction (2, -1)
A ray in the direction (-9, 20)
A vertex at (-7/62, -6/31)
A 2-dimensional polyhedron in QQ^2 defined as the convex hull of 1 vertex and 2 rays


In [4]:
import random
max([random.randint(0,9) for i in range(512)])

9